In [1]:
import pandas as pd


## Notes
Before even loading this CSV, I made a few changes.  I picked only properties with a sq ft > 1, which should remove all Lot/Land sales.  This sales data is from 9/1/2020 - 9/1/2021.  [Source of data](https://www.larimer.org/assessor/search#/data/?city=FORT%20COLLINS)

In [2]:
df = pd.read_csv("LC-SalesData-2021-09-06.csv")

In [3]:
df.head()

,accountno,parcelnb,saledt,locationaddress,locationcity,locationzipcode,subdivisionid,subdivisiondescr,nbhd,occdescr,bltasoccdesc,bltasid,residentialsf,bsmntsf,bsmntfinsf,garagesf,grossacres,yrblt,salep,value
0,R1600422,9614419390,2021-08-29,7214 AVONDALE RD,FORT COLLINS,80525,18513,"RIDGEWOOD HILLS PUD, THIRD FILING",19614.0,Single Family Residential,2 Story,8,2005,947.0,814.0,580.0,0.16,2001,615469.0,455800.0
1,R1295306,8606223014,2021-08-27,1219 OAK ISLAND CT,FORT COLLINS,80525,156651,OAKRIDGE VILLAGE PUD,19613.0,Townhouse,Townhouse One Story,61,1547,885.0,685.0,440.0,0.14,1989,495000.0,401500.0
2,R1377272,8606232004,2021-08-26,1318 SILK OAK DR,FORT COLLINS,80525,15669,OAKRIDGE VILLAGE PUD,19613.0,Single Family Residential,2 Story,8,2176,648.0,648.0,672.0,0.20,1994,595000.0,495000.0
3,R1556681,8830208084,2021-08-26,3220 HEARTHFIRE DR,FORT COLLINS,80524,1946,HEARTHFIRE PUD,18933.0,Single Family Residential,2 Story,8,3094,1554.0,NaN,726.0,0.54,2000,954000.0,692600.0
4,R1216503,8730416025,2021-08-26,1986 MASSACHUSETTS ST,FORT COLLINS,80525,1570,TIMBERLINE VILLAGE PUD,19724.0,Single Family Residential,Split Level,7,1256,NaN,NaN,440.0,0.16,1986,425000.0,323700.0


In [6]:
df.columns

Index(['accountno', ' parcelnb', ' saledt', ' locationaddress',
       ' locationcity', ' locationzipcode', ' subdivisionid',
       ' subdivisiondescr', ' nbhd', ' occdescr', ' bltasoccdesc', ' bltasid',
       ' residentialsf', ' bsmntsf', ' bsmntfinsf', ' garagesf', ' grossacres',
       ' yrblt', ' salep', ' value'],
      dtype='object')

Our columns have spaces in them, so we want to remove those spaces

In [10]:
df.rename(columns=lambda x: x.strip(), inplace=True)

In [11]:
df.columns

Index(['accountno', 'parcelnb', 'saledt', 'locationaddress', 'locationcity',
       'locationzipcode', 'subdivisionid', 'subdivisiondescr', 'nbhd',
       'occdescr', 'bltasoccdesc', 'bltasid', 'residentialsf', 'bsmntsf',
       'bsmntfinsf', 'garagesf', 'grossacres', 'yrblt', 'salep', 'value'],
      dtype='object')

In [22]:
len(df.loc[:,"subdivisiondescr"].unique())

251

There are 251 unique subdivisions among 500 total sales.  This made me wonder, with exactly 500 sales if I was really getting all of them.

In [44]:
from datetime import date

The larimer county data api takes in a very strange date format.  We need to duplicate it, because we're getting far more than 500 results, but only 500 are being returned.  We need to iterate through each month and get a month of data at a time.

Wed%20Jul%2001%202020%2000:00:00%20GMT-0600%20(Mountain%20Daylight%20Time)

In [34]:
today = date.today()

In [35]:
today

datetime.date(2021, 9, 6)

In [43]:
today.strftime( "%a %b %d %Y 00:00:00 GMT-0600 (Mountain Daylight Time)")

'Mon Sep 06 2021 00:00:00 GMT-0600 (Mountain Daylight Time)'

This is our date range.  We need to iterate through it to get our data.

In [46]:
pr = pd.period_range(start='2020-07',end='2021-09', freq='M')
prTuples = tuple([(period.month,period.year) for period in pr])
prTuples

((7, 2020),
 (8, 2020),
 (9, 2020),
 (10, 2020),
 (11, 2020),
 (12, 2020),
 (1, 2021),
 (2, 2021),
 (3, 2021),
 (4, 2021),
 (5, 2021),
 (6, 2021),
 (7, 2021),
 (8, 2021),
 (9, 2021))

In [47]:

# import urllib library
from urllib.request import urlopen
# import json
import json

url = "https://apps.larimer.org/api/assessor/?prop=saledata&city=FORT%20COLLINS&ecoarea=undefined&nbhd=&from_date={}&to_date={}&from_price=undefined&to_price=undefined&from_sf=1&to_sf=undefined&from_acres=undefined&to_acres=undefined&from_yrblt=undefined&to_yrblt=undefined&ressf=undefined&resyrblt=undefined&ressubno=undefined&style=undefined&resnbd=undefined&type=json&filename=LC-SalesData&viewfile=1"


In [ ]:
for i in prTuples:
    